In [517]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [518]:
Attack_list = ['Bruteforce_CWE-307','CVE-2012-2122','CVE-2014-0160','CVE-2017-7529','CVE-2018-3760',
               'CVE-2019-5418','EPS_CWE-434','PHP_CWE-434','SQL_Injection_CWE-89','ZipSlip']
Attack_list

['Bruteforce_CWE-307',
 'CVE-2012-2122',
 'CVE-2014-0160',
 'CVE-2017-7529',
 'CVE-2018-3760',
 'CVE-2019-5418',
 'EPS_CWE-434',
 'PHP_CWE-434',
 'SQL_Injection_CWE-89',
 'ZipSlip']

In [479]:
def make_csv(name): # .txt -> .csv converting function
    file_list = os.listdir("/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/"+name)
    file_list.remove('.DS_Store') #Mac의 경우만 존재하는 것 같음 Mac환경이 아니라면 이 row를 무방함
    print(len(file_list))
    base_dir = '/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/'+name+'/'#txt파일이 들어있는 폴더가 있는 dir
    base_csv_dir = '/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/'#CSV파일을 저장할 폴더
    for e in tqdm(file_list):
        dir = base_dir + e
        sample = pd.read_table(dir, header=None)
        tmp = list()
        for i in range (sample.shape[0]):
            x = sample[0][i]
            sp = x.split(' ', 8)
            tmp.append(sp)
        data = pd.DataFrame(tmp,columns=['event_num','event_time','cpu','user_uid','process','process_id','event_direction','event_type','event_argument'])
        data.reset_index(drop=True,inplace=True)
        file_name_csv = e.replace('.txt','.csv')
        save_dir = base_csv_dir + file_name_csv
        data.to_csv(save_dir)

In [241]:
for e in Attack_list: #.txt -> .csv converting part
    print(e)
    make_csv(e)

1221


100%|███████████████████████████████████████| 1221/1221 [04:23<00:00,  4.64it/s]


1079


100%|███████████████████████████████████████| 1079/1079 [04:04<00:00,  4.41it/s]


1071


...

100%|███████████████████████████████████████| 1112/1112 [05:47<00:00,  3.20it/s]


1078


100%|███████████████████████████████████████| 1078/1078 [06:06<00:00,  2.94it/s]


1063


100%|█████████████████████████████████████| 1063/1063 [1:07:01<00:00,  3.78s/it]


In [537]:
def runs_preprocessing(name): #run_file preprocessing function
    run_file = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/run/runs.csv')
    #runs.csv파일을 저는 CSV/Bruteforce_CWE-307/run/runs.csv와 같은 dir에 넣어놓았습니다.
    #run_file에 해당공격의 runs.csv파일을 읽어오게 하시면 됩니다.
    run_file.columns = ['image_name','scenario_name','is_executing_exploit','warmup_time',
                       'recording_time', 'exploit_start_time']
    run_file['image_name'] = run_file['image_name'].str.strip()
    run_file['scenario_name'] = run_file['scenario_name'].str.strip()
    run_file.to_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/run/runs.csv', index=False)
    #이부분도 원하시는 dir를 설정하여 저장하시면 됩니다.

In [526]:
for e in tqdm(Attack_list): # run_file preprocessing pa
    print(e)
    runs_preprocessing(e)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 159.70it/s]

Bruteforce_CWE-307
CVE-2012-2122
CVE-2014-0160
CVE-2017-7529
CVE-2018-3760
CVE-2019-5418
EPS_CWE-434
PHP_CWE-434
SQL_Injection_CWE-89
ZipSlip


In [538]:
def data_preprocessing(name): # data_preprocessing(labeling,...)
    file_list = os.listdir("/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/"+name)
    
    #CSV파일 리스트 읽어오기
    file_list.remove('.DS_Store') #Mac의 경우만 존재하는 것 같음
    file_list.remove('run') #저는 /CSV/run 폴더에 runs.csv파일을 넣어놓았기 때문에 디렉토리를 삭제
    print(len(file_list))
    base_csv_dir = '/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/'
    #CSV파일이 들어있는 폴더
    for e in tqdm(file_list):
        dir = base_csv_dir + e
        n = e.replace('.csv','')
        sample = pd.read_csv(dir, index_col=0)
        sample['event_time'] = sample['event_time'].str.slice(start=0, stop=8)
        sample['label'] = 0
        run_file = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+name+'/run/runs.csv')
        #해당 공격의 runs.csv파일을 읽어오게 하면 됩니다. dir 설정부탁
        if run_file['exploit_start_time'][run_file['scenario_name']==n].values[0] != -1:
            record_start = int(sample['event_time'][0][6:8])#time split only second
            exploit_start = run_file['exploit_start_time'][run_file['scenario_name']==n].values[0]
            tmp_time = record_start + exploit_start
            if tmp_time > 60:

                real_exploit_sec = tmp_time - 60
            else:
                real_exploit_sec = tmp_time
            sample.loc[sample['event_time'].str.slice(6,8) == str(real_exploit_sec),'label'] = 1
        sample.to_csv(dir)
            #그대로 저장

In [539]:
for e in Attack_list:
    print(e)
    data_preprocessing(e)

Bruteforce_CWE-307
1092


100%|███████████████████████████████████████| 1092/1092 [00:39<00:00, 27.57it/s]


CVE-2012-2122
1395


100%|███████████████████████████████████████| 1395/1395 [00:45<00:00, 30.47it/s]


CVE-2014-0160
1100


...

100%|███████████████████████████████████████| 1112/1112 [07:35<00:00,  2.44it/s]


SQL_Injection_CWE-89
1078


100%|███████████████████████████████████████| 1078/1078 [03:44<00:00,  4.80it/s]


ZipSlip
1063


100%|███████████████████████████████████████| 1063/1063 [28:22<00:00,  1.60s/it]


---------------------------------------------------------------------------------------------------------------
EDA Part

In [ ]:
file_list = os.listdir("/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/"+"Bruteforce_CWE-307")
file_list.remove('.DS_Store') #Mac의 경우만 존재하는 것 같음
file_list.remove('run') #저는 /CSV/run 폴더에 runs.csv파일을 넣어놓았기 때문에 디렉토리를 삭제
print(len(file_list))
tmp = list()
for e in tqdm(file_list):
    dir = "/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/"+"Bruteforce_CWE"+e
    sample = pd.read_csv(dir, index_col=0)
    arr = sample.array()
    tmp.append(sample)
data= pd.DataFrame(tmp,columns=['event_num','event_time','cpu','user_uid','process','process_id','event_direction','event_type','event_argument','label'])
print(data)
data.shape
    
    

---------------------------------------------------------------------------------------------------------------
Code test part

In [452]:
run_file = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'Bruteforce_CWE-307'+'/run/runs.csv')

In [403]:
sample = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'ex'+'/tiny_leakey_3874.csv', index_col = 0)

In [510]:
sample = pd.read_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'ex'+'/tiny_leakey_3874.csv', index_col = 0)
sample['event_time'] = sample['event_time'].str.slice(0,8)
sample['label']=0 # 
if run_file['exploit_start_time'][run_file['scenario_name']=='tiny_leakey_3874'].values[0] != -1:
    record_start = int(sample['event_time'][0][6:8])
    exploit_start = run_file['exploit_start_time'][run_file['scenario_name']=='tiny_leakey_3874'].values[0]
    tmp_time = record_start + exploit_start
    print(record_start)
    print(exploit_start)
    if tmp_time > 60:
        real_exploit_sec = tmp_time - 60
    else:
        real_exploit_sec = tmp_time
    print(real_exploit_sec)
    sample.loc[sample['event_time'].str.slice(6,8) == str(real_exploit_sec),'label'] = 1
    print(sample['label'].value_counts())
    sample.to_csv('/Users/cain/Desktop/Project/Anomaly_Detection_Project/LID-DS-2019/CSV/'+'ex'+'/haha.csv')
    

57
22
19
0    22735
1       82
Name: label, dtype: int64
